## Introduction

[Video Walkthrough](https://www.youtube.com/watch?v=Obbn15rZfvQ&list=PLypX5sYuDqvrqsXTw876gGHosCKvK_7QS&index=13)

This notebook demonstrates the implementation of a Retrieval-Augmented Generation (RAG) pipeline using KDB.AI and Large Language Models. By the end of this tutorial, you'll understand how to leverage vector databases and LLMs to create an effective RAG system.

### Setup and Dependencies
Install kdbai_client and import the necessary dependencies

##### Install Required Libraries

In [ ]:
# Install required libraries
!pip install llama-index fastembed kdbai_client openai

##### Import Dependencies

In [ ]:
import os
from getpass import getpass
import kdbai_client as kdbai
import time
from llama_index.core import Document, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
from fastembed import TextEmbedding
import openai
import textwrap

##### Connect to KDB.AI

In [ ]:
KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
    if "KDBAI_ENDPOINT" in os.environ
    else input("KDB.AI endpoint: ")
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
    if "KDBAI_API_KEY" in os.environ
    else getpass("KDB.AI API key: ")
)

KDB.AI endpoint: https://cloud.kdb.ai/instance/wrve8kwshj
KDB.AI API key: ··········


##### Initialize Embedding Model

In [ ]:
fastembed = TextEmbedding()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

### Data Preparation


##### Download Dataset
We'll use the Paul Graham Essay Dataset as our corpus.

In [ ]:
!llamaindex-cli download-llamadataset PaulGrahamEssayDataset --download-dir ./data

100% 1/1 [00:00<00:00,  2.72it/s]
Successfully downloaded PaulGrahamEssayDataset to ./data


### Create a KDB.AI session and table

In [ ]:
KDBAI_TABLE_NAME = "paul_graham"
session = kdbai.Session(endpoint=KDBAI_ENDPOINT, api_key=KDBAI_API_KEY)

# Drop existing table if it exists
try:
    session.table(KDBAI_TABLE_NAME).drop()
    time.sleep(5)
except kdbai.KDBAIException:
    pass

In [ ]:
# Define table schema

schema = dict(
    columns=[
        dict(name="text", pytype="bytes"),
        dict(
            name="embedding",
            vectorIndex=dict(type="flat", metric="L2", dims=384),
        ),
    ]
)

table = session.create_table(KDBAI_TABLE_NAME, schema)

#### Load and Parse Documents

In [ ]:
node_parser = SentenceSplitter(chunk_size=500, chunk_overlap=100)
essays = SimpleDirectoryReader(input_dir="./data/source_files").load_data()
docs = node_parser.get_nodes_from_documents(essays)
len(docs)

44

##### Generate Embeddings

In [ ]:
embedding_model = TextEmbedding()
documents = [doc.text for doc in docs]
embeddings = list(embedding_model.embed(documents))

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
print(documents[1])

So I'm not surprised I can't remember any programs I wrote, because they can't have done much. My clearest memory is of the moment I learned it was possible for programs not to terminate, when one of mine didn't. On a machine without time-sharing, this was a social as well as a technical error, as the data center manager's expression made clear.

With microcomputers, everything changed. Now you could have a computer sitting right in front of you, on a desk, that could respond to your keystrokes as it was running instead of just churning through a stack of punch cards and then stopping. [1]

The first of my friends to get a microcomputer built it himself. It was sold as a kit by Heathkit. I remember vividly how impressed and envious I felt watching him sitting in front of it, typing programs right into the computer.

Computers were expensive in those days and it took me years of nagging before I convinced my father to buy one, a TRS-80, in about 1980. The gold standard then was the Appl

#####  Insert Data into KDB.AI Table

In [ ]:
records_to_insert_with_embeddings = pd.DataFrame({
    "text": documents,
    "embedding": embeddings
})

In [ ]:
table.insert(records_to_insert_with_embeddings)

True

### RAG Implementation

##### Define Query and Generate Embedding

In [ ]:
query = "How does Paul Graham decide what to work on?"

In [ ]:
query_embedding = list(embedding_model.embed([query]))[0].tolist()

##### Perform Vector Search

In [ ]:
search_results = session.table(KDBAI_TABLE_NAME).search([query_embedding], n=10)
search_results_df = search_results[0]

In [ ]:
pd.set_option('display.max_colwidth', None)
print("Top Search Results Based on Query:", query)
df = pd.DataFrame(search_results_df)
df.head(2)

Top Search Results Based on Query: How do you decide what to work on?


,text,embedding,__nn_distance
0,"We knew this was coming, but it was still hard when it did.\n\nI kept working on YC till March, to help get that batch of startups through Demo Day, then I checked out pretty completely. (I still talk to alumni and to new startups working on things I'm interested in, but that only takes a few hours a week.)\n\nWhat should I do next? Rtm's advice hadn't included anything about that. I wanted to do something completely different, so I decided I'd paint. I wanted to see how good I could get if I really focused on it. So the day after I stopped working on YC, I started painting. I was rusty and it took a while to get back into shape, but it was at least completely engaging. [18]\n\nI spent most of the rest of 2014 painting. I'd never been able to work so uninterruptedly before, and I got to be better than I had been. Not good enough, but better. Then in November, right in the middle of a painting, I ran out of steam. Up till that point I'd always been curious to see how the painting I was working on would turn out, but suddenly finishing this one seemed like a chore. So I stopped working on it and cleaned my brushes and haven't painted since. So far anyway.\n\nI realize that sounds rather wimpy. But attention is a zero sum game. If you can choose what to work on, and you choose a project that's not the best one (or at least a good one) for you, then it's getting in the way of another project that is. And at 50 there was some opportunity cost to screwing around.\n\nI started writing essays again, and wrote a bunch of new ones over the next few months. I even wrote a couple that weren't about startups. Then in March 2015 I started working on Lisp again.\n\nThe distinctive thing about Lisp is that its core is a language defined by writing an interpreter in itself. It wasn't originally intended as a programming language in the ordinary sense. It was meant to be a formal model of computation, an alternative to the Turing machine. If you want to write an interpreter for a language in itself, what's the minimum set of predefined operators you need?","[-0.052486546, -0.037547894, 0.028428264, 0.002678531, 0.010856037, 0.01053419, -0.051966634, -0.015711516, 0.02491498, -0.037169933, 0.018490974, -0.004179084, -0.027491497, 0.01952365, 0.03600465, 0.011625322, -0.04530152, 0.0035254613, -0.030323114, -0.053392835, -0.038193867, -0.016321087, 0.013805427, -0.06196813, 0.04244484, 0.07093362, -0.03509179, -0.075523235, 0.009419392, -0.18652868, -0.012934241, 0.022660254, 0.0134734465, -0.0060935053, 0.031707864, 0.05483813, -0.059910905, 0.026537346, -0.021477206, 0.022409543, -0.0015035876, 0.025525393, -0.11022901, -0.032164246, 0.033352446, -0.018563112, -0.022256548, -0.04628748, 0.012898676, 0.029967379, -0.034196883, -0.053546622, 0.036861476, 0.050337255, -0.01440328, 0.02089807, 0.051477704, 0.05381708, 0.035958868, 0.011148146, -0.005329434, -0.023108445, -0.08358203, 0.039739255, -0.0154602025, 0.03253821, -0.006801028, -0.061871927, 0.050129626, 0.14079295, -0.0070907357, 0.019820116, -0.05166138, 0.06913528, 0.015589136, -0.04536847, 0.017244404, 0.010174067, 0.0384817, -0.024191776, 0.028073508, -0.008786634, 0.026387988, 0.015315114, -0.011378572, 0.01482849, -0.031060888, 0.0444626, 0.040066842, 0.0493386, 0.05094562, 0.020320805, 0.005881617, 0.013353571, -0.028635636, -0.042912528, -0.0044636712, 0.022763168, -0.019849092, 0.49997365, ...]",0.750161
1,"I hoped to lure Robert into working on it with me, but there I ran into a hitch. Robert was now a postdoc at MIT, and though he'd made a lot of money the last time I'd lured him into working on one of my schemes, it had also been a huge time sink. So while he agreed that it sounded like a plausible idea, he firmly refused to work on it.\n\nHmph. Well, I'd do it myself then. I recruited Dan Giffin, who had worked for Viaweb, and two undergrads who wanted summer jobs, and we got to work trying to build what it's now clear is about twenty companie

##### RAG Function Definition

In [ ]:
# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY_HERE"
    

def RAG(retrieved_data,prompt):
    messages = "Answer the following query in three sentences based on the context and only the context: " + "\n"
    messages += prompt + "\n"
    if len(retrieved_data) > 0:
        messages += "Context: " + "\n"
        for data in retrieved_data:
            messages += data + "\n"
    openai.api_key = os.environ["OPENAI_API_KEY"]
    response = openai.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {
            "role": "user",
            "content": [
                {"type": "text", "text": messages},
            ],
            },
        ],
        max_tokens=300,
    )
    content = response.choices[0].message.content
    return content

##### Execute RAG Pipeline

In [ ]:
# Utility Function for Text Wrapping

def print_wrapped(text, width=80):
    wrapper = textwrap.TextWrapper(width=width)
    word_list = wrapper.wrap(text=text)
    for line in word_list:
        print(line)

In [ ]:
print("Query:", query)

print_wrapped(RAG(search_results_df["text"],query))

Query: How do you decide what to work on?
The person decides what to work on by following their interests and the urge to
try something completely different, as evidenced by their shift from working
intensively at YC to painting and later writing essays and working on Lisp. They
are guided by the desire to dive deeply into subjects or projects that capture
their curiosity, even if that means deviating from their previous path or
professional expertise. The decision-making process involves a mix of self-
reflection on past choices, seeking new challenges, and the willingness to
explore areas where they can be truly independent and potentially make lasting
contributions.


### Drop Table To Conserve Resources

In [ ]:
table.drop()